In [38]:
from pathlib import Path
import Bio
from Bio.PDB import PDBList
import shutil
from Bio.PDB import PDBParser
import mdtraj
from Bio.Align.Applications import ClustalOmegaCommandline
import subprocess
from Bio import SeqIO
import matplotlib.pyplot as plt
from os import path, system
from engens.core.CrystalUtils import *
import requests
import pandas as pd

In [29]:
# get PDBRenum files
def get_pdb_bioassembly_pdbrenum(pdb_id, dst):
    cmd_get_pdbrenum = "wget http://dunbrack3.fccc.edu/PDBrenum/output_PDB_assembly/{}_renum.pdb1.gz".format(pdb_id)
    ret = os.system(cmd_get_pdbrenum)
    
    cmd_get_pdbrenum = "mv {}_renum.pdb1.gz {}/{}_renum.pdb1.gz".format(pdb_id, dst, pdb_id)
    ret = os.system(cmd_get_pdbrenum)
    cmd_ungz = "gzip -d {}/{}_renum.pdb1.gz".format(dst, pdb_id)
    ret = os.system(cmd_ungz)
    res_file = "{}/{}_renum.pdb1".format(dst, pdb_id)
    return res_file

In [30]:
# fix files with PDBFixer
def fix_pdb_files(input_file, output_file):
    pdbfixer_command = "pdbfix.py -i {} -o {}".format(input_file,output_file)
    os.system(pdbfixer_command)
    return output_file

In [39]:
# for the given PDB entry get associated entities
def rscb_entities_from_entries(pdb_ids):
    pdb_ids_str = "[\""+"\",\"".join(pdb_ids)+"\"]"

    # QUERY - get the entity ids for the given entry ids
    query = """query {
      entries(entry_ids: """+pdb_ids_str+"""){
        polymer_entities {
          rcsb_id
          rcsb_polymer_entity_container_identifiers {
            reference_sequence_identifiers {
              database_accession
              database_name
            }
          }
        }
      }
    }"""  
    
    url = 'https://data.rcsb.org/graphql'
    r = requests.post(url, json={'query': query})
    print(r.status_code)
    
    if r.status_code == 200:
        json_data = r.json()

        #reformat dictionary
        ref_json_data = []
        for i, elem in enumerate(json_data["data"]["entries"]):
            pdb_id = pdb_ids[i]
            elem_polymers = elem["polymer_entities"]
            for ep in elem_polymers:
                rcsb_id = ep["rcsb_id"]
                if ep["rcsb_polymer_entity_container_identifiers"]["reference_sequence_identifiers"] is not None:
                    for seq_id in ep["rcsb_polymer_entity_container_identifiers"]["reference_sequence_identifiers"]:
                        assecion = seq_id["database_accession"]
                        db_name = seq_id["database_name"]
                        ref_json_data.append({"pdb_id":pdb_id, \
                                              "entity_id": rcsb_id,\
                                              "accession": assecion,\
                                              "database": db_name})
                else:
                    ref_json_data.append({"pdb_id":pdb_id, \
                                          "entity_id": rcsb_id,\
                                          "accession": None,\
                                          "database": None})
                    
        results_df = pd.DataFrame(ref_json_data)
        return results_df
    else:
        print("HTTP Request error {} for RSCB polymer_entities query".format(r.status_code))
        return None
    
    

In [42]:
#For the given entity ID - get the chain number
def rscb_polymer_chains_info(entity_ids):
    
    entity_id_str = "[\"" + "\",\"".join(entity_ids)+"\"]"
    
    # QUERY - get the entry ids for all the above to collect asym_ids
    query = """
    query {
      polymer_entities(entity_ids:"""+entity_id_str+""") {
        rcsb_id
        rcsb_entity_source_organism {
          ncbi_taxonomy_id
          ncbi_scientific_name
        }
        rcsb_cluster_membership {
          cluster_id
          identity
        }
        rcsb_polymer_entity_container_identifiers{
          asym_ids
        }
      }
    }
    """
    
    url = 'https://data.rcsb.org/graphql'
    r = requests.post(url, json={'query': query})
    print(r.status_code)
    
    if r.status_code == 200:
        json_data = r.json()
        polymer_entity_data = json_data['data']["polymer_entities"]
        entity_instance_connection = {"entity_id": [], "asym_ids":[]}
        for elem in polymer_entity_data:
            rcsb_id = elem["rcsb_id"]
            identifiers = elem["rcsb_polymer_entity_container_identifiers"]
            entity_instance_connection["asym_ids"].append(identifiers["asym_ids"])
            entity_instance_connection["entity_id"].append(rcsb_id)
        entity_instance_connection_df = pd.DataFrame(entity_instance_connection)
        return entity_instance_connection_df
    else:
        print("HTTP Request error {} for RSCB polymer_entities query".format(r.status_code))
        return None

In [61]:
def rscb_get_author_instance_info(instance_ids):
    
    entity_instance_ids = "[\"" + "\",\"".join(instance_ids)+"\"]"
    
    # QUERY - find the author chain id and author sequence mapping
    query = """
    query {
      polymer_entity_instances(instance_ids: """+entity_instance_ids+""") {
        rcsb_id
        rcsb_polymer_entity_instance_container_identifiers {
          asym_id
          auth_asym_id,
          auth_to_entity_poly_seq_mapping
        }
      }
    }

    """
    
    url = 'https://data.rcsb.org/graphql'
    r = requests.post(url, json={'query': query})
    print(r.status_code)
    
    if r.status_code == 200:
        json_data = r.json()
        polymer_entity_instances_data = json_data['data']["polymer_entity_instances"]
        entity_instances_mappings = {"instance_id": [], "asym_id":[], "auth_asym_id":[], "auth_seq_map":[]}
        for elem in polymer_entity_instances_data:
            rcsb_id = elem["rcsb_id"]
            identifiers = elem["rcsb_polymer_entity_instance_container_identifiers"]
            entity_instances_mappings["instance_id"].append(rcsb_id)
            entity_instances_mappings["asym_id"].append(identifiers["asym_id"])
            entity_instances_mappings["auth_asym_id"].append(identifiers["auth_asym_id"])
            entity_instances_mappings["auth_seq_map"].append(identifiers["auth_to_entity_poly_seq_mapping"])
        entity_instance_mapping_df = pd.DataFrame(entity_instances_mappings)
        return entity_instance_mapping_df
    else:
        print("HTTP Request error {} for RSCB polymer_entity_instances query".format(r.status_code))
        return None

In [70]:
def uniprot_get_details(uniprot_ids):
    uniprot_details = {"accession_id":[], 
                       "id":[],
                       "full_name":[],
                       "seq" : []}
    uniprot_accession_url = "https://www.ebi.ac.uk/proteins/api/proteins/"
    
    for uni_id in uniprot_ids:
        accession_query = uniprot_accession_url+uni_id
        result_uniprot_details = requests.get(accession_query)
        if result_uniprot_details.status_code == 200:
            res_json = result_uniprot_details.json()
            uniprot_details["accession_id"].append( res_json["accession"] )
            uniprot_details["id"].append( res_json["id"] )
            uniprot_details["full_name"].append( res_json['protein']['recommendedName']['fullName']['value'] )
            uniprot_details["seq"].append( res_json['sequence']['sequence'] )
            
        else:
            print("Uniprot query failed: response "+result_uniprot_details.status_code)
            return None
    return pd.DataFrame(uniprot_details)

In [79]:
def get_pdb_metadata(pdb_ids):
    #pdb_id to entity (splitting chains, geting uniprot data)
    print("Fetching metadata for PDB entries - pdb id and related entity ids")
    entity_df = rscb_entities_from_entries(pdb_ids)
    #entity to instance (mapping chains to uniprot ids)
    print("Fetching metadata for PDB entries - entity ids and related uniprot ids")
    entity_ids = list(entity_df["entity_id"].unique())
    entity_instance_connection_df = rscb_polymer_chains_info(entity_ids)
    entity_instance_connection_df["first_asym_id"] = entity_instance_connection_df["asym_ids"].apply(lambda x: x[0])
    entity_df = entity_df.merge(entity_instance_connection_df, how="left", on="entity_id")
    entity_df["instance_id"] = entity_df["pdb_id"].apply(lambda x: x.upper())+"."+entity_df["first_asym_id"]
    instance_ids = list(entity_df["instance_id"].unique())
    #get author sequence mappings for each instance
    print("Fetching metadata for PDB entries - instance ids and related sequences")
    entity_instance_mapping_df = rscb_get_author_instance_info(instance_ids)
    entity_instance_mapping_df["entity_id"] = entity_instance_mapping_df.merge(entity_df, how="left", on="instance_id").entity_id
    #get uniprot metadata
    print("Fetching metadata for UniProt ids")
    uniprot_ids = entity_df["accession"].unique()
    uniprot_details = uniprot_get_details(uniprot_ids)
    return (entity_df, uniprot_details)

In [80]:
entity_df, uniprot_df = get_pdb_metadata(pdbIds)

Fetching metadata for PDB entries - pdb id and related entity ids
200
Fetching metadata for PDB entries - entity ids and related uniprot ids
200
Fetching metadata for PDB entries - instance ids and related sequences
200
Fetching metadata for UniProt ids


In [84]:
from IPython.display import display_html 


df1_styler = entity_df[["pdb_id", "entity_id", "accession", "database"]].style.set_table_attributes("style='display:inline'").set_caption('PDB metadata')
df2_styler = uniprot_df[["accession_id", "id", "full_name"]].style.set_table_attributes("style='display:inline'").set_caption('UniProt metadata')
space = "\xa0" * 10
display_html(df1_styler._repr_html_()+ space  + df2_styler._repr_html_(), raw=True)

PDB metadata pdb_id entity_id accession database 
 
 0 
 2rd0 
 2RD0_1 
 P42336 
 UniProt 
 
 
 1 
 2rd0 
 2RD0_2 
 P27986 
 UniProt 
 
 
 2 
 3hhm 
 3HHM_1 
 P42336 
 UniProt 
 
 
 3 
 3hhm 
 3HHM_2 
 P27986 
 UniProt 
 
 
 4 
 3hiz 
 3HIZ_1 
 P42336 
 UniProt 
 
 
 5 
 3hiz 
 3HIZ_2 
 P27986 
 UniProt 
 
 
 6 
 4jps 
 4JPS_1 
 P42336 
 UniProt 
 
 
 7 
 4jps 
 4JPS_2 
 P27986 
 UniProt 
 
            UniProt metadata accession_id id full_name 
 
 0 
 P42336 
 PK3CA_HUMAN 
 Phosphatidylinositol 4,5-bisphosphate 3-kinase catalytic subunit alpha isoform 
 
 
 1 
 P27986 
 P85A_HUMAN 
 Phosphatidylinositol 3-kinase regulatory subunit alpha

In [87]:
#MAP EACH UNIPROT ACCESSION TO ANOTHER 
#example: same chain from different species -> map all corresponding chains to one selected uniprot id
#example: different variants of the same chain -> map all variants to one selected uniprot id
#default: each chain is mapped to it
my_accession_mapping = dict(zip(uniprot_df["accession_id"].unique(), uniprot_df["accession_id"].unique()))
##MAKE YOUR CHANGES HERE
#For Example
#my_accession_mapping["P42336"] = "P27986"
my_accession_mapping

{'P42336': 'P42336', 'P27986': 'P27986'}

In [56]:
#check that each structure has the corresponding uniprot component
for uniprot_id in uniprot_ids:
    for pdb_id in pdb_ids:
        cnt = entity_df[(entity_df.accession == uniprot_id) & (entity_df.pdb_id == pdb_id)].shape[0]
        if cnt > 0

1
1
1
1
1
1
1
1


# List of PDB files/ids - as input to the workflow

In [88]:
# Get the PDB ID-s of your files

# Our example:
# PI3K pdb IDs - from the paper: https://doi.org/10.1016/j.jmb.2020.09.002 
# Supplementary material table S2.

pdbIds =  "2rd0 3hhm 3hiz 4jps \
5swg 5swo 5swp 5swr 5swt 5sx8 5sx9 5sxa \
5sxb 5sxc 5sxd 5sxe 5sxf 5sxi 5sxj 5sxk".split()
'''
\
5uk8 5ukj 5ul1 5xgh 5xgi 5xgj 6nct \
4a55 2y3a \
5dxu 5m6u 5t8f 5ubt 5vlr 6g6w 6pyr 6pyu".split()
'''

print("{} total PI3K pdb ids used here.".format(len(pdbIds)))

20 total PI3K pdb ids used here.


0

## Step 1  - download PDBs

In [ ]:
# Make the directory to  store the files in 
# Change this to any name you'd like
pdir = "./PI3Ks_pdbs"
Path(pdir).mkdir(parents=True, exist_ok=True)

# -----------------STEP 1.1 - download PDBs------------------------ #
# Initialize CrystalUtils - it will autimatically (if no file_names provieded):
# Download PDBs (renumbered) 
# Fix them
#crystal_utils = CrystalUtils(pdb_codes = pdbIds, dst_folder = pdir)
for pdb_id in pdbIds:
    res_file = get_pdb_bioassembly_pdbrenum(pdb_id, pdir)
    fix_pdb_files(res_file, res_file[:-5]+"_fixed.pdb")

In [ ]:

# -----------------STEP 1.2 - download associated Uniprot IDs (for alignment purposes)------------------------ #



In [17]:
get_pdb_bioassembly_pdbrenum("3hhm", ".")

1024
512


'./3hhm_renum.pdb1'

0

In [5]:
# -----------------STEP 1.2 - extract protein sequences------------------------ #
# Fasta file name - place to store the sequence
crystal_utils.extract_protein_sequence()


FileNotFoundError: [Errno 2] No such file or directory: 'PI3Ks_pdbs/structure_output/2rd0_fixed.pdb'

In [5]:
# -----------------STEP 1.3 - compute MSA with clustal omega------------------------ #
# ----------------- and extract maximum common substructure ------------------------ #
print("Running MSA of the sequences")
crystal_utils.performMSA()

Running MSA of the sequences
Running MSA of the sequences
Found PI3Ks_pdbs/sequence_output/sequence_aligned.fasta
Continuous region #1 found starting in AA range 14-239
Continuous region #2 found starting in AA range 256-318
Continuous region #3 found starting in AA range 327-414
Continuous region #4 found starting in AA range 425-499
Continuous region #5 found starting in AA range 523-858
Continuous region #6 found starting in AA range 869-935
Continuous region #7 found starting in AA range 948-965
Visualization od MSA and common regions: PI3Ks_pdbs/sequence_output/sequence_aligned_regions.html
Extracting substructures
Extracting common regions from each file (backbone)


Extracting common regions from each file (backbone): 100%|██████████| 4/4 [00:00<00:00, 9383.23it/s]

File exists: PI3Ks_pdbs/structure_output/2rd0_fixed_bbstrip.pdb
File exists: PI3Ks_pdbs/structure_output/3hhm_fixed_bbstrip.pdb
File exists: PI3Ks_pdbs/structure_output/3hiz_fixed_bbstrip.pdb
File exists: PI3Ks_pdbs/structure_output/4jps_fixed_bbstrip.pdb
Converting to trajectory bb_traj.xtc


Extracting common regions from each file (full residues)


Extracting common regions from each file (full residues): 100%|██████████| 4/4 [00:00<00:00, 8077.62it/s]

File exists: PI3Ks_pdbs/structure_output/2rd0_fixedresstrip.pdb
File exists: PI3Ks_pdbs/structure_output/3hhm_fixedresstrip.pdb
File exists: PI3Ks_pdbs/structure_output/3hiz_fixedresstrip.pdb
File exists: PI3Ks_pdbs/structure_output/4jps_fixedresstrip.pdb
Converting to trajectory resstrip_traj.xtc


In [13]:
# ----------------- visualize and inspect the alignment! ------------------------ #
from IPython.display import IFrame
IFrame('PI3Ks_pdbs/sequence_output/sequence_aligned_regions.html', width=1050, height=200)

# Now - continue with the normal workflow: but add the list of pdb files as an argument to EnGens

- remember to align the trajectory (align = True when constructing EnGen)
- make sure your binding_site_selstr is something that is generalizable to different possibly mutated residues
- same for the featurization (do not use all atom featurization - since different residues have different number of atoms)
- do not use TICA/HDE
- do not use VAMP nets to select features

these are only for use with time series data (MDs)

In [15]:
import glob
from engens.core.EnGens import *

# input files:
pdb_files_processed = glob.glob(path.join(crystal_utils.dst_structure, "*resstrip.pdb"))
# any random bbstrip file
top_loc = path.join(crystal_utils.dst_structure, crystal_utils.pdb_codes[0]+'_fixed_bbstrip.pdb')
# backbone - common residue trajectory
traj_loc = path.join(crystal_utils.dst_structure, "bb_traj.xtc")
# input files - containing full common residues
input_files = pdb_files_processed
structure_names = [name[name.rfind("/")+1:name.rfind("/")+5] for name in input_files]

engen = EnGen(traj_loc, top_loc, cryst_pdb_list = True, file_names = input_files, structure_names = structure_names, align=True)


Loading files (might take a while): 100%|██████████| 4/4 [00:01<00:00,  2.14it/s]


# Workflow 1 - extract features from the PDB files

**Input:** reference PDB and trajectory


**Output:** featurized trajectory
<hr>
Steps:

1. Load reference PDB and trajectory in the EnGen object
3. Provide set of featurizations of interest (or use default)
4. Evaluate different featurization (optional)
5. Choose the best featurization
6. Extract those features

In [16]:
# required imports 
import engens.core.FeatureSelector as fs
import pickle
import mdshare
import mdtraj
import numpy as np
import nglview
from IPython.display import Javascript, display
import json

### Step 1 - load the structure and trajectory

Provide the path to the files with the reference trajectory and topology.
(You can use any format that <a fref = https://mdtraj.org/1.9.4/api/generated/mdtraj.load.html> mdtraj.load </a> will take as input).

Optionally, provide a subset of the structure that you will use for featurization (e.g. binding site) as a <a href=https://mdtraj.org/1.9.4/atom_selection.html> atom selection string </a> or a list of atom indices.


In [18]:
nglwidget = select_residues_nglview(top_loc)
nglwidget

ThemeManager()

NGLWidget(gui_style='ngl')

In [19]:
## Option 4 - continue selection 1
selection = None
display(Javascript(js_script))

<IPython.core.display.Javascript object>

In [ ]:
## Option 4 - continue selection 2
if not selection is None and len(selection) > 0:
    binding_site_selstr = get_selstring(selection)
    #binding_site_selstr = "(10 <= resid) and (resid <= 50)" 
    engen = EnGen(traj_loc, top_loc, binding_site_selstr, align = True)

#------------------------end of options----------------------------#

In [20]:

#visualize the trajectory (optional - if trajectory too large, skip this step)
nglwidget = engen.show_animated_traj()
nglwidget.clear_representations()
nglwidget.add_ball_and_stick()
nglwidget.center()
nglwidget

NGLWidget(max_frame=48)

In [21]:
nglview.write_html("tmp.html", [nglwidget], frame_range=(0, 20))

### Step 2 - select different featurizations

Here we select ways to featurize the trajectory. Any PyEmma <a href = http://www.emma-project.org/latest/api/generated/pyemma.coordinates.featurizer.html> trajectory featurization </a> can be used in this step and any of the parameters of the respective featurizations can be provided. Users can also use the default initialization which includes three sets of features: (1) amino-acid pairwise distances; (2) torsion angles and (3) amino-acid pairwise distances with the torsion angels.

In [ ]:
# remove any existing featurizers
engen.reset_featurizers()
# initialize default features 
engen.init_featurizers_default()
description = engen.describe_featurizers()
print(description)

In [ ]:
# Split chains to groups and assign group distances as features

tmp_traj = mdtraj.load(engen.ref)
df_top = tmp_traj.topology.to_dataframe()[0]

groups = {}
groups_list = []
for chain in df_top.chainID.unique():
    group_resSeq = df_top[df_top.chainID == chain].serial
    groups_list.append(list(group_resSeq))
    groups[chain] = list(group_resSeq)

In [ ]:
df_top

In [ ]:
# remove any existing featurizers
engen.reset_featurizers()

#center of mass and torsion angles
feat = {
    "add_group_mindist": {"group_definitions": groups_list}
}

#add the respective features to the engen structure
engen.add_featurizer(feat)
description = engen.describe_featurizers()
print(description)

### Step 3 - evaluate the featurizations

This step is optional - we recommend evaluating the featurizations and picking the best using PyEmma's implementation of <a href=http://www.emma-project.org/latest/tutorials/notebooks/00-pentapeptide-showcase.html#Feature-selection> VAMP approach </a>.

This helps you choose a set of features with which to proceed to the next Workflow.

### Not an option for crystal structure input!!

### Step 4 - pick the featurization

We suggest using the featurization which gives you the highest VAMP2 score from the analysis above. To do so, run the cell below.

In [ ]:
#apply features
engen.apply_featurizations()
#print possible features
print(engen.describe_featurizers())
#select the number of the desired feature
feat_num = 0
# initialize selector
featsel = fs.UserFeatureSelection(feat_num, engen)
#select the feature
featsel.select_feature()

### Step 5 - save the results as input for Workflow2 - dimensionality reduction

In [ ]:
# save the results for next workflow
with open("wf1_resulting_EnGen.pickle", "wb") as file:
    pickle.dump(engen, file, -1)